In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('/kaggle/input/ner-dataset/NER_Dataset.csv')

In [2]:
df

,Sentence_ID,Word,POS,Tag
0,Sentence: 1,"['Thousands', 'of', 'demonstrators', 'have', '...","['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '..."
1,Sentence: 10,"['Iranian', 'officials', 'say', 'they', 'expec...","['JJ', 'NNS', 'VBP', 'PRP', 'VBP', 'TO', 'VB',...","['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '..."
2,Sentence: 100,"['Helicopter', 'gunships', 'Saturday', 'pounde...","['NN', 'NNS', 'NNP', 'VBD', 'JJ', 'NNS', 'IN',...","['O', 'O', 'B-tim', 'O', 'O', 'O', 'O', 'O', '..."
3,Sentence: 1000,"['They', 'left', 'after', 'a', 'tense', 'hour-...","['PRP', 'VBD', 'IN', 'DT', 'NN', 'JJ', 'NN', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,Sentence: 10000,"['U.N.', 'relief', 'coordinator', 'Jan', 'Egel...","['NNP', 'NN', 'NN', 'NNP', 'NNP', 'VBD', 'NNP'...","['B-geo', 'O', 'O', 'B-per', 'I-per', 'O', 'B-..."
...,...,...,...,...
47954,Sentence: 9995,"['Opposition', 'leader', 'Mir', 'Hossein', 'Mo...","['NNP', 'NN', 'NNP', 'NNP', 'NNP', 'VBZ', 'VBN...","['O', 'O', 'O', 'B-per', 'I-per', 'O', 'O', 'O..."
47955,Sentence: 9996,"['On', 'Thursday', ',', 'Iranian', 'state', 'm...","['IN', 'NNP', ',', 'JJ', 'NN', 'NNS', 'VBN', '...","['O', 'B-tim', 'O', 'B-gpe', 'O', 'O', 'O', 'O..."
47956,Sentence: 9997,"['Following', 'Iran', ""'s"", 'disputed', 'June'...","['VBG', 'NNP', 'POS', 'JJ', 'NNP', 'CD', 'NNS'...","['O', 'B-geo', 'O', 'O', 'B-tim', 'I-tim', 'O'..."
47957,Sentence: 9998,"['Since', 'then', ',', 'authorities', 'have', ...","['IN', 'RB', ',', 'NNS', 'VBP', 'VBN', 'JJ', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


# EDA
Investigate how many Tags are in the dataset

In [3]:
import ast 

# Current data has list form read in literal. Change this to list form
for i, row in df.iterrows():
    df.loc[i, 'Word'] = ast.literal_eval(df.loc[i, 'Word'])
    df.loc[i, 'Tag'] = ast.literal_eval(df.loc[i, 'Tag'])

unique_tags = set()

def add_unique_tags(row):
    taglist = row['Tag']
    for tag in taglist:
        # Add the tag only if not in the unique_tags set
        if tag not in unique_tags:
            unique_tags.add(tag)

df.apply(add_unique_tags, axis=1)
unique_tags

{'B-art',
 'B-eve',
 'B-geo',
 'B-gpe',
 'B-nat',
 'B-org',
 'B-per',
 'B-tim',
 'I-art',
 'I-eve',
 'I-geo',
 'I-gpe',
 'I-nat',
 'I-org',
 'I-per',
 'I-tim',
 'O'}

In [4]:
# Let's hardcode the tags to reorder
tags = ['O', 'B-art', 'I-art', 'B-eve', 'I-eve', 'B-geo', 'I-geo', 'B-gpe', 'I-gpe', 
        'B-nat', 'I-nat', 'B-org', 'I-org', 'B-per', 'I-per', 'B-tim', 'I-tim']

In [5]:
import sklearn
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Word'], df['Tag'], test_size=0.2, random_state=42)

In [6]:
X_train[0]

['Thousands',
 'of',
 'demonstrators',
 'have',
 'marched',
 'through',
 'London',
 'to',
 'protest',
 'the',
 'war',
 'in',
 'Iraq',
 'and',
 'demand',
 'the',
 'withdrawal',
 'of',
 'British',
 'troops',
 'from',
 'that',
 'country',
 '.']

# Make Vocabulary
(Note: 'O' tagged tokens will be excluded from the vocabulary)

In [7]:
def make_vocab(x_data, y_labels):
    '''
    Args:
        :x_data: (train_data_length, sentence_length)
        :y)data: (train_data_length, 1)
    '''
    vocab = set()
    
    # Loop through each sentence
    for sentence, tags in zip(x_data, y_labels):
        # Loop through each token
        for token, tag in zip(sentence, tags):
            if tag != 'O':
                vocab.add(token)
                
    vocab = list(vocab)
    return vocab

In [8]:
vocab_train = make_vocab(X_train, y_train)
vocab_test = make_vocab(X_test, y_test)

In [9]:
print(f"Length of vocabulary of train dataset: {len(vocab_train)}")
print(f"Length of vocabulary of test dataset:  {len(vocab_test)}")

Length of vocabulary of train dataset: 12061
Length of vocabulary of test dataset:  5686


In [10]:
# Checking how many tokens from test vocab is NOT in train vocab
count = 0
for i in range(len(vocab_test)):
    if vocab_test[i] not in vocab_train:
        count += 1

print(f"{count} Tokens from vocab_test are not in vocab_train")

1498 Tokens from vocab_test are not in vocab_train


# Encoding
Encoding each token to integers
* Special Tokens x->y mapping:
    * '\<PAD\>' -> '\<PAD\>'
    * '\<UNK\>' -> 'O'

In [11]:
def word2idx(special_tokens, vocab):
    num_spec_tokens = len(special_tokens)
    # Word2idx
    word_to_idx = {spec: idx for idx, spec in enumerate(special_tokens)} # Adding special tokens
    word_to_idx.update({token: (idx + num_spec_tokens) for idx, token in enumerate(vocab)}) # Adding non-special vocab tokens
    # idx2Word
    idx_to_word = {idx: token for token, idx in word_to_idx.items()} # Key-value swapping
    
    return word_to_idx, idx_to_word

In [12]:
token2idx, idx2token = word2idx(['<PAD>', '<UNK>'], vocab_train)
tag2idx, idx2tag = word2idx(['<PAD>'], tags)

In [13]:
tag2idx

{'<PAD>': 0,
 'O': 1,
 'B-art': 2,
 'I-art': 3,
 'B-eve': 4,
 'I-eve': 5,
 'B-geo': 6,
 'I-geo': 7,
 'B-gpe': 8,
 'I-gpe': 9,
 'B-nat': 10,
 'I-nat': 11,
 'B-org': 12,
 'I-org': 13,
 'B-per': 14,
 'I-per': 15,
 'B-tim': 16,
 'I-tim': 17}

In [14]:
# Encoding Tokenized data
def encode_tokens(tokenized_data, token2idx, unknown_token='<UNK>'):
    '''
    Args:
        :tokenized_data: (dataset_length, sequence_length)
        :token2idx: {token : idx}
        :unknown_token: The token that the unknown tokens(not in vocab e.g. token2idx) will be mapped to 
    Returns:
        :encoded_data: listof(listof(int))
    '''
    encoded_data = []
    
    for sent in tokenized_data:
        encoded_sent = []
        for token in sent:
            try:
                encoded_sent.append(token2idx[token])
            except KeyError: # If not found, it means unkown token
                encoded_sent.append(token2idx[unknown_token])
        # Adding encoded sentence to the final output
        encoded_data.append(encoded_sent)
    
    return encoded_data

In [15]:
# X dataset
enc_X_train = encode_tokens(X_train, token2idx, '<UNK>')
enc_X_test = encode_tokens(X_test, token2idx, '<UNK>')
# y dataset
enc_y_train = encode_tokens(y_train, tag2idx, 'O')
enc_y_test = encode_tokens(y_test, tag2idx, 'O')

In [16]:
#enc_X_train

# Determine Context Length

In [17]:
import matplotlib.pyplot as plt

def set_sent_pad(encode_x, context_length, report=False):
    if report:
        print(f'훈련 데이터셋 최대 길이: {max(len(exam) for exam in encode_x)}')
        print(f'훈련 데이터셋 평균 길이: {sum(map(len, encode_x)) / len(encode_x):.2f}')

        plt.hist([len(exam) for exam in encode_x], bins=50)
        plt.xlabel('length of samples')
        plt.ylabel('number of samples')
        plt.show()

    def below_th_len(encode_x, context_length):
        cnt = 0
        for sent in encode_x:
            if(len(sent) <= context_length):
                cnt +=1
        
        print(f'데이터셋 문장 길이가 {context_length} 이하 데이터 비율: '+
              f'{cnt / len(encode_x)*100:.2f}%')
        
    below_th_len(encode_x, context_length)
    
set_sent_pad(enc_X_train, 40)

데이터셋 문장 길이가 40 이하 데이터 비율: 98.40%


# Padding

In [18]:
def pad_seq(dataset, context_length):
    padded = np.zeros((len(dataset), context_length), dtype=int)
    for idx, sent in enumerate(dataset):
        if len(sent) != 0:
            padded[idx, :len(sent)] = np.array(sent)[:context_length]
    return padded

In [19]:
CONTEXT_LENGTH = 40
# X datasets
pad_X_train = pad_seq(enc_X_train, CONTEXT_LENGTH)
pad_X_test = pad_seq(enc_X_test, CONTEXT_LENGTH)
# y datasets
pad_y_train = pad_seq(enc_y_train, CONTEXT_LENGTH)
pad_y_test = pad_seq(enc_y_test, CONTEXT_LENGTH)

In [20]:
print(f"{pad_X_train.shape}")
print(f"{pad_y_train.shape}")

(38367, 40)
(38367, 40)


# DataLoader

In [21]:
import torch
from torch.utils.data import TensorDataset, DataLoader

def set_dataloader(x_data, y_label, bs, training=False):
    tensor_x = torch.tensor(x_data, dtype=torch.int64)
    tensor_y = torch.tensor(y_label, dtype=torch.int64)
    
    dataset = TensorDataset(tensor_x, tensor_y)
    
    shuffle=False
    if training:
        shuffle=True
    
    dataloader = DataLoader(dataset, shuffle=shuffle, batch_size=bs)
    
    return dataloader

In [22]:
BATCH_SIZE = 48

train_dataloader = set_dataloader(pad_X_train, pad_y_train, bs=BATCH_SIZE, training=True)
test_dataloader = set_dataloader(pad_X_test, pad_y_test, bs=BATCH_SIZE, training=False)

# Model (RNN)

In [23]:
from torch import nn

class RNN_Tagger(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, 
                 num_tags=18, num_layers=2, embed_matrix=None):
        super().__init__()
        # Embedding Layer
        self.embed = nn.Embedding(vocab_size, embed_dim)
        # If pretrained-embedding is used
        if embed_matrix is not None:
            # Transferring weights
            self.embed.weight = nn.Parameter(
                torch.tensor(embed_matrix, dtype=torch.float32)
            )
            self.embed.weight.requires_grad = True
        # RNN Layer
        self.rnn = nn.RNN(input_size=embed_dim,
                          hidden_size=hidden_dim,
                          num_layers=num_layers,
                          bidirectional=True,
                          batch_first=True)
        # Classifier Layer 
        # Since bidirectional, input size is hidden_dim*2
        self.classifier = nn.Linear(hidden_dim*2, num_tags)
        
    def forward(self, x):
        '''
        Input:
            :x: (batch_size, context_length) 
        '''
        emb_x = self.embed(x) # :emb_x: (batch_size, context_length, embed_dim)
        output, hidden = self.rnn(emb_x) # :output: (batch_size, context_length, hidden_dim*2)
        logits = self.classifier(output)
        return logits

In [24]:
# Hyperparameters & Model variables
VOCAB_SIZE = len(vocab_train)
EMBEDDING_DIM = 100
HIDDEN_DIM = 500
NUM_TAGS = len(tag2idx)
NUM_LAYERS = 2

In [25]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

RNN_NERtagger = RNN_Tagger(vocab_size=VOCAB_SIZE, embed_dim=EMBEDDING_DIM,
                           hidden_dim=HIDDEN_DIM, num_tags=NUM_TAGS, num_layers=NUM_LAYERS)
RNN_NERtagger = RNN_NERtagger.to(DEVICE)

# Loss and Optimizer

In [26]:
import torch.optim as optim

# Don't count '<PAD>' tokens when calculating loss
loss_fn = nn.CrossEntropyLoss(ignore_index=0)
# Opimizer
optimizer = optim.Adam(RNN_NERtagger.parameters(), lr=0.001)

# Train

In [27]:
from tqdm import tqdm

def train(model, dataloader, 
          loss_fn, optimizer):
    
    model.train()
    total, correct = 0, 0
    train_loss, train_acc = 0.0, 0.0
    
    for X, y in tqdm(dataloader):
        X, y = X.to(DEVICE), y.to(DEVICE)
        pred_logits = model(X)
        # Reshape pred and traget before feeding into loss_fn
        batch_size, context_length, num_tags = pred_logits.size()
        pred_logits = pred_logits.reshape(-1, num_tags) # -> (bs*context_length, num_tags)
        y = y.reshape(-1) # (batch_size, context_length) -> (batch_size*context_length)
        
        # Calculate Loss
        loss = loss_fn(pred_logits, y)
        # optimizer zero_grad
        optimizer.zero_grad()
        # loss backward
        loss.backward()
        # Optimizer step
        optimizer.step()
        
        train_loss += loss.item()
        
        # Calculating Accuracy
        y_pred = torch.argmax(pred_logits, axis=1)
        # Masking to not count <PAD> as correct
        IGNORE_IDX = 0 # <PAD> = 0
        mask = (y != IGNORE_IDX)
        total += mask.sum().item()
        correct += y_pred.eq(y.view_as(y_pred)).masked_select(mask).sum().item()
    
    train_loss /= len(dataloader)
    train_acc = 100 * (correct / total)
    return train_loss, train_acc

In [28]:
def evaluate(model, dataloader, loss_fn):
    model.eval()
    correct, total = 0, 0
    test_loss, test_acc = 0.0, 0.0
    
    for X, y in tqdm(dataloader):
        X, y = X.to(DEVICE), y.to(DEVICE)
        pred_logits = model(X)
        
        # Reshape pred and traget before feeding into loss_fn
        batch_size, context_length, num_tags = pred_logits.size()
        pred_logits = pred_logits.reshape(-1, num_tags) # -> (bs*context_length, num_tags)
        y = y.reshape(-1) # (batch_size, context_length) -> (batch_size*context_length)
        
        # Calculate Loss
        loss = loss_fn(pred_logits, y)
        test_loss += loss.item()
        
        #Calculating Accuracy
        y_pred = torch.argmax(pred_logits, axis=1)
        # Masking to not count <PAD> as correct
        IGNORE_IDX = 0 # <PAD> = 0
        mask = (y != IGNORE_IDX)
        total += mask.sum().item()
        correct += y_pred.eq(y.view_as(y_pred)).masked_select(mask).sum().item()
    
    test_loss /= len(dataloader)
    test_acc = 100 * (correct / total)
    return test_loss, test_acc

In [29]:
import copy
import time

def model_train(model, train_dataloader, test_dataloader,
                loss_fn, optimizer, num_epochs):
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())
    
    for epoch in range(1, num_epochs + 1):
        start = time.time()
        train_loss, train_acc = train(model, train_dataloader, loss_fn, optimizer)
        test_loss, test_acc = evaluate(model, test_dataloader, loss_fn)
        
        if test_acc > best_acc:
            best_acc = test_acc
            best_model_wts = copy.deepcopy(model.state_dict())
        
        end = time.time()
        print(f"------------ epoch {epoch} ------------")
        print(f"Train loss: {train_loss:.4f} | Train acc: {train_acc:.2f}%")
        print(f"Val loss: {val_loss:.4f} | Val acc: {val_acc:2f}%")
        print(f"Time taken: {(end - start) / 60:.0f}min {(end - start) % 60:.0f}s")
        
    # Finalize the best model
    model.load_state_dict(best_model_wts)
    return model

In [30]:
# torch.backends.cudnn.enabled = False

# RNN_tagger_model = model_train(RNN_NERtagger, 
#                                train_dataloader, 
#                                test_dataloader, 
#                                loss_fn, 
#                                optimizer, 
#                                10)